In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

**Reading Data**

In [ ]:
df=pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
df

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

**Treating Outliers**

In [ ]:
for i in df.iloc[:,3:].columns:
    sns.boxplot(x=df[i],data=df,palette='rainbow')
    
    plt.show()

In [ ]:
l=[3,4,5,10,11,12,13,18,19,20]
def Outlier(data):
    for column in data.iloc[:,l].columns:
        if data[column].dtype!='str':
            Q1=np.percentile(data[column],25)
            Q3=np.percentile(data[column],75)
            IQR= Q3-Q1
            lower=Q1-(1.5* IQR)
            upper=Q3+(1.5* IQR)
            data.loc[:,column] =np.where(data[column].values>upper,upper,data[column].values)
            data.loc[:,column] =np.where(data[column].values<lower,lower,data[column].values)
    return data
 
df1=Outlier(df)
df1.head()

In [ ]:
for i in df1.iloc[:,3:].columns:
    sns.boxplot(x=df1[i],data=df1,palette='rainbow')
    
    plt.show()

In [ ]:
# heatmap

corr = df1.corr()
plt.figure(figsize=(14,14))
sns.heatmap(corr, annot=True, fmt= '.2f',annot_kws={'size': 15}, cmap= 'coolwarm')
plt.show()
print(corr)

In [ ]:
# Plot a histogram

df.price.hist(bins=30, alpha=0.5)
plt.show()

**Splitting Data**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=df1.drop(['id','date','price','zipcode'],axis=1)
y=df1['price']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=2020)

**Building A Model**

In [ ]:
#Scaling the data

from sklearn.preprocessing import MinMaxScaler

min_max=MinMaxScaler()

X_train_transformed=min_max.fit_transform(X_train)

X_test_transformed=min_max.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

clf=KNeighborsRegressor(n_neighbors=5,metric='euclidean')

In [ ]:
clf.fit(X_train_transformed,y_train)

In [ ]:
pred1=clf.predict(X_test_transformed)
pred1

In [ ]:
train_pred1=clf.predict(X_train_transformed)
train_pred1

**Evaluating The Model**

In [ ]:
#R2

from sklearn.metrics import r2_score
print('R^2:',r2_score(y_test,pred1))
print('R^2:',r2_score(y_train,train_pred1))

In [ ]:
#MSE

from sklearn.metrics import mean_squared_error
print('MSE:',mean_squared_error(y_test,pred1))
print('MSE:',mean_squared_error(y_train,train_pred1))

In [ ]:
#RMSE

print('RMSE:',np.sqrt(mean_squared_error(y_test,pred1)))
print('RMSE:',np.sqrt(mean_squared_error(y_train,train_pred1)))

In [ ]:
#MAE

from sklearn.metrics import mean_absolute_error
print('MAE:',mean_absolute_error(y_test,pred1))
print('MAE:',mean_absolute_error(y_train,train_pred1))